# Train LeJEPA on TPU (Google Colab)

⚠️ **Note**: TPU training is more complex than GPU. Use GPU unless you need TPU's speed for large-scale training.

**When to use TPU:**
- Training on 50K+ samples
- Batch size 128+
- GPUs unavailable

**Limitations:**
- FAISS doesn't work on TPU (uses CPU fallback)
- Some PyTorch ops not supported
- More complex debugging

## 1. Setup TPU

In [ ]:
# Check TPU availability
import os

# Set TPU environment
if 'TPU_NAME' in os.environ:
    print(f"✅ TPU detected: {os.environ['TPU_NAME']}")
else:
    print("⚠️  No TPU found. Go to Runtime → Change runtime type → TPU")
    raise RuntimeError("Please enable TPU in Colab settings")

In [ ]:
# Install TPU-specific packages
!pip install -q cloud-tpu-client
!pip install -q torch-xla>=2.0.0

# Install other dependencies
!pip install -q transformers>=4.45.0 sentence-transformers>=3.0.0 datasets
!pip install -q accelerate
!pip install -q lejepa || pip install -q git+https://github.com/rbalestr-lab/lejepa.git

print("✅ Packages installed")

## 2. Imports (TPU-specific)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sentence_transformers import SentenceTransformer
import lejepa
import numpy as np
from tqdm.auto import tqdm
import json
from pathlib import Path

# TPU-specific imports
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

print("✅ Imports successful")
print(f"PyTorch: {torch.__version__}")
print(f"XLA: {torch_xla.__version__}")

## 3. Model Definition (Same as GPU)

In [ ]:
class IsotropicGaussianEncoder(nn.Module):
    """Same model as GPU version - TPU compatible"""
    
    def __init__(self, output_dim=512, freeze_early_layers=True):
        super().__init__()
        
        print("Loading EmbeddingGemma-300M...")
        self.base = SentenceTransformer(
            'google/embeddinggemma-300m',
            trust_remote_code=True
        )
        
        self.projection = nn.Sequential(
            nn.Linear(768, 768 * 2),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(768 * 2, output_dim)
        )
        
        for param in self.base.parameters():
            param.requires_grad = True
        
        if freeze_early_layers:
            frozen = 0
            for name, param in self.base.named_parameters():
                if any(f'encoder.layer.{i}.' in name for i in range(4)):
                    param.requires_grad = False
                    frozen += 1
            print(f"Froze {frozen} parameters")
        
        total = sum(p.numel() for p in self.parameters())
        trainable = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f"Total: {total:,}, Trainable: {trainable:,}")
        
        self.output_dim = output_dim
    
    def encode(self, texts, batch_size=32, show_progress=False):
        base_emb = self.base.encode(
            texts,
            batch_size=batch_size,
            convert_to_tensor=True,
            show_progress_bar=show_progress,
            normalize_embeddings=True
        )
        gaussian_emb = self.projection(base_emb)
        return gaussian_emb
    
    def forward(self, texts):
        return self.encode(texts, show_progress=False)

print("✅ Model class defined")

## 4. Load Data (Same as GPU)

In [ ]:
from datasets import load_dataset

print("Loading training data...")
dataset = load_dataset(
    'sentence-transformers/msmarco-hard-negatives',
    'triplet',
    split='train',
    streaming=True
)

num_samples = 5000
data = []

for i, example in enumerate(dataset):
    if i >= num_samples:
        break
    data.append({
        'query': example['query'],
        'positive': example['positive'],
        'negative': example['negative']
    })
    if (i + 1) % 1000 == 0:
        print(f"  Loaded {i + 1}...")

train_size = int(0.9 * len(data))
train_data = data[:train_size]
val_data = data[train_size:]

print(f"✅ Train: {len(train_data)}, Val: {len(val_data)}")

In [ ]:
class TripletDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def collate_fn(batch):
    return {
        'queries': [item['query'] for item in batch],
        'positives': [item['positive'] for item in batch],
        'negatives': [item['negative'] for item in batch]
    }

# TPU works best with larger batches
batch_size = 128  # Increase for TPU (GPU: 16)

train_dataset = TripletDataset(train_data)
val_dataset = TripletDataset(val_data)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=0  # TPU doesn't need workers
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0
)

print(f"✅ Batch size: {batch_size} (larger for TPU efficiency)")

## 5. TPU-Specific Training Function

In [ ]:
def train_on_tpu():
    """Main training function for TPU"""
    
    # Get TPU device
    device = xm.xla_device()
    print(f"Using device: {device}")
    
    # Initialize model
    model = IsotropicGaussianEncoder(output_dim=512).to(device)
    
    # LeJEPA loss
    sigreg = lejepa.multivariate.SlicingUnivariateTest(
        univariate_test=lejepa.univariate.EppsPulley(num_points=17),
        num_slices=1024
    ).to(device)
    
    # Optimizer
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=1e-5,
        weight_decay=0.05
    )
    
    # Scheduler
    num_epochs = 5
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=num_epochs * len(train_loader)
    )
    
    lambda_contrastive = 1.0
    lambda_isotropy = 0.01
    
    Path('checkpoints').mkdir(exist_ok=True)
    best_val_loss = float('inf')
    
    print("\n" + "="*60)
    print("Starting TPU Training!")
    print("="*60)
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        
        # Training
        model.train()
        
        # TPU-specific: wrap dataloader
        para_loader = pl.ParallelLoader(train_loader, [device])
        train_loader_tpu = para_loader.per_device_loader(device)
        
        total_loss = 0
        
        for batch_idx, batch in enumerate(tqdm(train_loader_tpu, desc="Training")):
            # Encode triplets
            q_emb = model(batch['queries'])
            pos_emb = model(batch['positives'])
            neg_emb = model(batch['negatives'])
            
            # Losses
            pos_dist = torch.norm(q_emb - pos_emb, dim=1)
            neg_dist = torch.norm(q_emb - neg_emb, dim=1)
            contrastive_loss = torch.relu(pos_dist - neg_dist + 1.0).mean()
            
            all_emb = torch.cat([q_emb, pos_emb, neg_emb], dim=0)
            isotropy_loss = sigreg(all_emb)
            
            loss = lambda_contrastive * contrastive_loss + lambda_isotropy * isotropy_loss
            
            # Backward
            optimizer.zero_grad()
            loss.backward()
            
            # TPU-specific: sync gradients
            xm.optimizer_step(optimizer)
            scheduler.step()
            
            total_loss += loss.item()
            
            if batch_idx % 10 == 0:
                xm.master_print(f"  Batch {batch_idx}: loss={loss.item():.4f}")
        
        avg_train_loss = total_loss / len(train_loader)
        xm.master_print(f"\nTrain Loss: {avg_train_loss:.4f}")
        
        # Validation
        model.eval()
        val_loss = 0
        
        para_loader_val = pl.ParallelLoader(val_loader, [device])
        val_loader_tpu = para_loader_val.per_device_loader(device)
        
        with torch.no_grad():
            for batch in tqdm(val_loader_tpu, desc="Validation"):
                q_emb = model(batch['queries'])
                pos_emb = model(batch['positives'])
                neg_emb = model(batch['negatives'])
                
                pos_dist = torch.norm(q_emb - pos_emb, dim=1)
                neg_dist = torch.norm(q_emb - neg_emb, dim=1)
                contrastive_loss = torch.relu(pos_dist - neg_dist + 1.0).mean()
                
                all_emb = torch.cat([q_emb, pos_emb, neg_emb], dim=0)
                isotropy_loss = sigreg(all_emb)
                
                loss = lambda_contrastive * contrastive_loss + lambda_isotropy * isotropy_loss
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        xm.master_print(f"Val Loss: {avg_val_loss:.4f}")
        
        # Save best model (only on master TPU core)
        if xm.is_master_ordinal() and avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            # Move to CPU before saving
            model_cpu = model.cpu()
            torch.save({
                'epoch': epoch,
                'model_state_dict': model_cpu.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, 'checkpoints/best_model.pt')
            model.to(device)  # Move back to TPU
            xm.master_print("  ⭐ Saved best model!")
    
    xm.master_print("\n✅ Training complete!")

print("✅ Training function defined")

## 6. Run Training on TPU

In [ ]:
# Run training
# Note: TPU uses all 8 cores by default
train_on_tpu()

print("\n" + "="*60)
print("Training finished! Model saved to checkpoints/best_model.pt")
print("="*60)

## 7. Download Model

In [ ]:
from google.colab import files

# Save final model
final_path = 'gaussian_embeddinggemma_final.pt'

# Load checkpoint and save just the model state dict
checkpoint = torch.load('checkpoints/best_model.pt', map_location='cpu')
torch.save(checkpoint['model_state_dict'], final_path)

print(f"✅ Model saved: {final_path}")

# Download
files.download(final_path)

print("\n📥 Download complete!")
print("Next: Save to /Users/ctn/src/ctn/ragcun/data/embeddings/")

## 🎯 TPU Training Notes

### **Key Differences from GPU:**

1. **Batch Size:**
   - GPU: 16-32 optimal
   - TPU: 128+ optimal (TPUs love large batches!)

2. **DataLoader:**
   - Must use `ParallelLoader` wrapper
   - `num_workers=0` (TPU handles parallelism)

3. **Optimizer Step:**
   - Use `xm.optimizer_step()` instead of `optimizer.step()`
   - Syncs gradients across TPU cores

4. **Device:**
   - `device = xm.xla_device()` instead of `cuda`

5. **Saving:**
   - Move model to CPU before saving
   - Only save on master ordinal

### **Performance:**

- **Speed:** ~2-3x faster than T4 GPU for large batches
- **Cost:** Free in Colab (same as GPU)
- **Complexity:** Higher (more code, harder debugging)

### **Recommendation:**

Use **GPU (T4/V100)** unless:
- ✅ Training on 50K+ samples
- ✅ Using batch size 128+
- ✅ GPUs are unavailable

For this project (5K samples, batch 16-32): **GPU is simpler and sufficient!**